In [ ]:
# Import all necessary libraries
import numpy as np
import pandas as pd
from scipy.optimize import fmin
import matplotlib.pyplot as plt
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
from aquacrop.utils import prepare_weather, get_filepath

In [ ]:
# Locate weather file
filepath=get_filepath('d:/Research/sandbox/iot_extra/db/climate_data.txt')

weather_data = prepare_weather(filepath)
weather_data

In [ ]:
# Simulation period
sim_start = '2022/01/01'
sim_end = '2023/12/31'
plant_date = '11/15'
harvest_date = '04/01'

In [ ]:
# Proccess soil
soil_data = pd.read_csv('d:/Research/sandbox/iot_extra/db/soil_data.csv', sep=',')

In [ ]:
# Add soil profile
local_soil = Soil(soil_type='ClayLoam', dz=[0.3]*2+[0.4]+[1.0])
for index, layer in soil_data.iterrows():
    local_soil.add_layer_from_texture(
        thickness=layer["thickness"],
        Sand=layer["sand"],
        Clay=layer["clay"],
        OrgMat=layer["om"],
        # Assumption full penetrability, unable to calculate for now
        penetrability=100
    ) 
local_soil.profile

In [ ]:
# Water Content
# Layers are assumed to receive different amount of water initially 
multiTawWC = InitialWaterContent(wc_type = 'Prop',
                                method = 'Depth',
                                depth_layer= [0.3, 0.6, 1, 2],
                                value = ["FC", "SAT", "WP", "WP"])


In [ ]:
# Add crop profile 
potato = Crop(
    c_name='Potato',
    planting_date=plant_date,
    #harvest_date=harvest_date,
)

In [ ]:
# Irrigation method
# Uses drip irrigation
irr_mgnt = IrrigationManagement(
    irrigation_method=1, 
    SMT=[99, 99, 99, 99],
    MaxIrrSeason=750
    )

In [ ]:
# Combine into aquacrop model and specify start and end simulation date
model = AquaCropModel(sim_start_time=sim_start,
                      sim_end_time=sim_end,
                      weather_df=weather_data,
                      soil=local_soil,
                      crop=potato,
                      initial_water_content=multiTawWC,
                      irrigation_management=irr_mgnt, 
                      off_season=True)
model.run_model(till_termination=True)


In [ ]:
# Optimize moisture thresholds for irrigation 

# Model runner 
def run_model(smt, max_irr_season, year1, year2): 
    """
    Function to run the model and return soil moisture  
    """
    weather_filepath=get_filepath(
        'd:/Research/sandbox/iot_extra/db/climate_data.txt'
        )
    soil_data = pd.read_csv(
        'd:/Research/sandbox/iot_extra/db/soil_data.csv', sep=','
        )
    
    weather_data = prepare_weather(weather_filepath)
    
    potato = Crop(c_name='Potato', planting_date=plant_date)
    
    local_soil = Soil(soil_type='ClayLoam', dz=[0.3]*2+[0.4]+[1.0])
    for index, layer in soil_data.iterrows():
        local_soil.add_layer_from_texture(
            thickness=layer["thickness"],
            Sand=layer["sand"],
            Clay=layer["clay"],
            OrgMat=layer["om"],
            # Assumption full penetrability, unable to calculate for now
            penetrability=100
        ) 
    
    multiTawWC = InitialWaterContent(
        wc_type = 'Prop',
        method = 'Depth',
        depth_layer= [1, 2, 3, 4],
        value = ["FC", "SAT", "WP", "WP"]
        )
    
    irr_mgnt = IrrigationManagement(
        irrigation_method=1, 
        SMT=smt, 
        MaxIrrSeason=max_irr_season
        )
    
    model = AquaCropModel(
        sim_start_time=f"{year1}/01/01",
        sim_end_time=f"{year2}/12/31",
        weather_df=weather_data,
        soil=local_soil,
        crop=potato,
        initial_water_content=multiTawWC,
        irrigation_management=irr_mgnt
        )
    
    model.run_model(till_termination=True)
    
    return model.get_simulation_results()
    

def evaluate(smts,max_irr_season,test=False):
    """
    function to run model and calculate reward (yield) for given set of soil moisture targets
    """
    # Run model
    out = run_model(smts, max_irr_season, year1=2022, year2=2023)
    # Get yields and total irrigation
    yld = out['Dry yield (tonne/ha)'].mean()
    tirr = out['Seasonal irrigation (mm)'].mean()


    reward=yld

    # Return either the negative reward (for the optimization)
    # Or the yield and total irrigation (for analysis)
    if test:
        return yld,tirr,reward
    else:
        return -reward

def get_starting_point(num_smts,max_irr_season,num_searches):
    """
    find good starting threshold(s) for optimization
    """

    # Get random SMT's
    x0list = np.random.rand(num_searches,num_smts)*100
    rlist=[]
    # evaluate random SMT's
    for xtest in x0list:
        r = evaluate(xtest,max_irr_season,)
        rlist.append(r)

    # Save best SMT
    x0=x0list[np.argmin(rlist)]
    
    return x0

def optimize(num_smts,max_irr_season,num_searches=100):
    """ 
    optimize thresholds to be profit maximising
    """
    # Get starting optimization strategy
    x0=get_starting_point(num_smts,max_irr_season,num_searches)
    # Run optimization
    res = fmin(evaluate, x0,disp=0,args=(max_irr_season,))
    # Reshape array
    smts= res.squeeze()
    # Evaluate optimal strategy
    return smts

from tqdm.autonotebook import tqdm # Progress bar

opt_smts=[]
yld_list=[]
tirr_list=[]
for max_irr in tqdm(range(0,500,50)):
    

    # Find optimal thresholds and save to list
    smts= optimize(4,max_irr)
    opt_smts.append(smts)

    # Save the optimal yield and total irrigation
    yld,tirr,_= evaluate(smts,max_irr,True)
    yld_list.append(yld)
    tirr_list.append(tirr)
    
# Create plot
fig,ax=plt.subplots(1,1,figsize=(13,8))

# Plot results
ax.scatter(tirr_list,yld_list)
ax.plot(tirr_list,yld_list)

# Labels
ax.set_xlabel('Total Irrigation (ha-mm)',fontsize=18)
ax.set_ylabel('Dry yield (tonne/ha)',fontsize=18)
ax.set_xlim([-20,600])
ax.set_ylim([2,15.5])

# Annotate with optimal thresholds
bbox = dict(boxstyle="round",fc="1")
offset = [15,15,15, 15,15,-125,-100,  -5, 10,10]
yoffset= [0,-5,-10,-15, -15,  0,  10,15, -20,10]
for i,smt in enumerate(opt_smts):
    smt=smt.clip(0,100)
    ax.annotate('(%.0f, %.0f, %.0f, %.0f)'%(smt[0],smt[1],smt[2],smt[3]),
                (tirr_list[i], yld_list[i]), xytext=(offset[i], yoffset[i]), textcoords='offset points',
                bbox=bbox,fontsize=12)

In [ ]:
# Calculate yearly precipitation 
weather_data['Year'] = weather_data['Date'].dt.year
yearly_precip = weather_data.groupby('Year')['Precipitation'].sum().reset_index()
#yearly_stats = weather_data.groupby('Year')['Precipitation'].agg(['sum', 'mean', 'count']).reset_index()
yearly_precip

In [ ]:
# Log results
pd.set_option('display.max_rows', 20)  # or a large number like 1000
pd.set_option('display.max_columns', None)  # or a large number like 100
#model._outputs.water_flux.head(100)
#model._outputs.water_storage.head()
#model._outputs.crop_growth
#model._outputs.final_stats
model._outputs.final_stats.head(20)

In [ ]:
# Export results
model._outputs.water_flux.IrrDay.head(10)
date_range = pd.date_range(start=sim_start, end=sim_end)
irr_df = pd.DataFrame({
    'Year': date_range.year,
    'Month': date_range.month,
    'Day': date_range.day
})
irr_day_values = model._outputs.water_flux['IrrDay']
irr_df['IrrDay'] = irr_day_values.values[:len(date_range)]
irr_df.to_csv('db/irr_schedule.csv',sep=',',index=False)
